<a href="https://colab.research.google.com/github/Mr22XX/AI_Model/blob/main/AiPrivacy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import StandardScaler

# Contoh Data Transaksi: [Jumlah Transaksi, Waktu dalam Sehari (0-24)]
transactions = np.array([
    [500, 14], [600, 16], [550, 12], [3000, 3],  # Data Normal
    [450, 10], [700, 18], [680, 20], [50, 2],   # Data Normal
    [10000, 23], [12000, 1]  # Transaksi mencurigakan
])

# Normalisasi data
scaler = StandardScaler()
X = torch.tensor(scaler.fit_transform(transactions), dtype=torch.float32)

# Model Autoencoder untuk Deteksi Anomali
class AnomalyDetector(nn.Module):
    def __init__(self):
        super(AnomalyDetector, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(2, 4),
            nn.ReLU(),
            nn.Linear(4, 2),
            nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.Linear(2, 4),
            nn.ReLU(),
            nn.Linear(4, 2)
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

# Inisialisasi model
model = AnomalyDetector()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Training Autoencoder
num_epochs = 500
for epoch in range(num_epochs):
    optimizer.zero_grad()
    outputs = model(X)
    loss = criterion(outputs, X)  # Loss berdasarkan rekonstruksi error
    loss.backward()
    optimizer.step()

    if (epoch+1) % 100 == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

# Deteksi Anomali
with torch.no_grad():
    reconstructed = model(X)
    loss_per_point = torch.mean((X - reconstructed) ** 2, dim=1).numpy()

# Tentukan ambang batas anomali
threshold = np.percentile(loss_per_point, 95)
anomalies = np.where(loss_per_point > threshold)[0]

print("\nTransaksi Mencurigakan pada Indeks:", anomalies)


Epoch [100/500], Loss: 1.0000
Epoch [200/500], Loss: 1.0000
Epoch [300/500], Loss: 1.0000
Epoch [400/500], Loss: 1.0000
Epoch [500/500], Loss: 1.0000

Transaksi Mencurigakan pada Indeks: [9]
